In [1]:
#| default_exp train_gpt2

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [4]:
#| export
from dataclasses import dataclass
import torch, math
import torch.nn as nn
import torch.nn.functional as F

In [5]:
#| export
@dataclass
class GPTConfig:
    block_size: int = 1024
    vocab_size: int = 50257
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768
    

In [6]:
#| export
class CausalSelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1

        self.n_head = config.n_head
        self.n_embd = config.n_embd

        self.register_buffer('bias', torch.tril(torch.ones(config.block_size, config.block_size))
                             .view(1, 1, config.block_size, config.block_size), persistent=False)

    def forward(self, x):
        B, T, C = x.size()

        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)

        k = k.view(B, T, self.n_head, C//self.n_head).transpose(1, 2)
        q = q.view(B, T, self.n_head, C//self.n_head).transpose(1, 2)
        v = v.view(B, T, self.n_head, C//self.n_head).transpose(1, 2)

        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        y = att @ v
        y = y.transpose(1, 2).contiguous().view(B, T, C)

        y = self.c_proj(y)
        return y
        

In [7]:
#| export
class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu = nn.GELU(approximate='tanh')
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x
        

In [8]:
#| export
class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x
        

In [9]:
#| export
class GPT(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        self.transformer.wte.weight = self.lm_head.weight

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            std = 0.02
            if hasattr(module, 'NANOGPT_SCALE_INIT'):
                std *= (2 * self.config.n_layer) ** -0.5
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.size()
        assert T <= self.config.block_size, f'Cannot forward sequence of length {T}, block size is set to {config.block_size}'

        pos = torch.arange(0, T, dtype=torch.long, device=idx.device)
        pos_emb = self.transformer.wpe(pos)
        tok_emb = self.transformer.wte(idx)
        x = pos_emb + tok_emb

        for block in self.transformer.h:
            x = block(x)
            
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

    @classmethod
    def from_pretrained(cls, model_type):
        from transformers import GPT2LMHeadModel
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        print(f'loading weights from pretrained gpt: {model_type}')

        config_args = {
            'gpt2':        dict(n_layer=12, n_head=12, n_embd=768),
            'gpt2-medium': dict(n_layer=24, n_head=16, n_embd=1024),
            'gpt2-large':  dict(n_layer=36, n_head=20, n_embd=1280),
            'gpt2-xl':     dict(n_layer=48, n_head=25, n_embd=1600),
        }[model_type]
        config_args['vocab_size'] = 50257
        config_args['block_size'] = 1024

        config = GPTConfig(**config_args)
        model = GPT(config)
        sd = model.state_dict()
        sd_keys = sd.keys()

        model_hf = GPT2LMHeadModel.from_pretrained('gpt2')
        sd_hf = model_hf.state_dict()
        sd_hf_keys = sd_hf.keys()

        
        assert len(sd_keys) == len(sd_hf_keys), f'mismatched keys: {len(sd_keys)} != {len(sd_hf_keys)}'

        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        for k in sd_hf_keys:
            if any(k.endswith(o) for o in transposed):
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])
                
        return model
        

In [10]:
#| export
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = 'mps'

print(f'Using device : {device}')

Using device : cuda


In [267]:
# get a data batch
with open('input.txt', 'r') as f:
    text = f.read()

import tiktoken

enc = tiktoken.get_encoding('gpt2')
text = text[:1000]
tokens = enc.encode(text)

B, T = 4, 32
buf = torch.tensor(tokens[: B*T + 1])
buf = buf.to(device)

x = buf[:-1].view(B, T)
y = buf[1:].view(B, T)

In [11]:
#| export
import tiktoken

class DataLoaderLite:

    def __init__(self, B, T):
        self.B, self.T = B, T

        with open('input.txt', 'r') as f:
            text = f.read()
    
        enc = tiktoken.get_encoding('gpt2')
        tokens = enc.encode(text)
        self.tokens = torch.tensor(tokens)

        print(f'loaded {len(self.tokens)} tokens.')
        print(f'1 epoch = {len(self.tokens)//(B*T)} batches')

        self.current_position = 0

    def next_batch(self):
        B, T = self.B, self.T
        buf = self.tokens[self.current_position:self.current_position+(B*T)+1]
        x = buf[:-1].view(B, T)
        y = buf[1:].view(B, T)

        self.current_position += B*T
        if self.current_position + (B*T) > len(self.tokens):
            self.current_position = 0
        return x, y
        

In [12]:
#| export
import time

In [19]:
torch.set_float32_matmul_precision('high')

In [13]:
torch.manual_seed(1337)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1337)
    

In [14]:
#| export
model = GPT(GPTConfig())
model = model.to(device)

In [15]:
#| export
train_dataloader = DataLoaderLite(B=8, T=1024)

loaded 338025 tokens.
1 epoch = 41 batches


In [23]:
#| export
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, betas=(0.9, 0.95), eps=1e-8)

for i in range(50):
    t0 = time.time()
    x, y = train_dataloader.next_batch()
    x, y = x.to(device), y.to(device)
    
    optimizer.zero_grad()
    logits, loss = model(x, y)
    loss.backward()
    norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    torch.cuda.synchronize()
    
    t1 = time.time()
    dt = (t1 - t0) * 1000
    tokens_per_sec = (train_dataloader.B * train_dataloader.T)/(t1 - t0)
    print(f"Step {i} | loss: {loss.item()} | norm: {norm:.4f} | dt: {dt:.2f}ms | tok/sec: {tokens_per_sec:.2f}")
    

Step 0, loss: 5.874366283416748, dt: 4465.33ms, tok/sec: 1834.58
Step 1, loss: 6.306445598602295, dt: 1398.98ms, tok/sec: 5855.70
Step 2, loss: 5.931770324707031, dt: 1517.44ms, tok/sec: 5398.58
Step 3, loss: 5.832102298736572, dt: 1526.39ms, tok/sec: 5366.93
Step 4, loss: 5.833522796630859, dt: 1523.87ms, tok/sec: 5375.79
Step 5, loss: 5.7070841789245605, dt: 1512.84ms, tok/sec: 5414.97
Step 6, loss: 5.909891128540039, dt: 1353.19ms, tok/sec: 6053.85
Step 7, loss: 5.744501113891602, dt: 1519.99ms, tok/sec: 5389.49
Step 8, loss: 5.736121654510498, dt: 1520.79ms, tok/sec: 5386.67
Step 9, loss: 5.747751235961914, dt: 1520.33ms, tok/sec: 5388.30
Step 10, loss: 5.789073467254639, dt: 1518.98ms, tok/sec: 5393.08
Step 11, loss: 5.5597453117370605, dt: 1350.18ms, tok/sec: 6067.34
Step 12, loss: 5.496979713439941, dt: 1517.07ms, tok/sec: 5399.87
Step 13, loss: 5.989266395568848, dt: 1517.74ms, tok/sec: 5397.49
Step 14, loss: 5.890313148498535, dt: 1521.46ms, tok/sec: 5384.29
Step 15, loss: 5.9

## Generation

In [ ]:
# model = GPT.from_pretrained('gpt2')
model = GPT(GPTConfig())
model.eval()
model = model.to(device)

In [230]:
num_return_sequences = 5
max_length = 30

In [232]:
import tiktoken

enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode("Hello, I'm a language model,")
tokens = torch.tensor(tokens, dtype=torch.long)
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)
x = tokens.to(device)

In [233]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

while x.size(1) < max_length:
    with torch.no_grad():
        
        logits = model(x)
        logits = logits[:, -1]

        probs = F.softmax(logits, dim=-1)
        topk_probs, topk_indices = torch.topk(probs, k=50, dim=1)

        ix = torch.multinomial(topk_probs, 1)
        xcol = topk_indices.gather(1, ix)
        
        x = torch.cat((x, xcol), dim=1)
        

In [234]:
for i in range(num_return_sequences):
    tokens = x[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print('> ', decoded)
    

>  Hello, I'm a language model, WiFiirableiewicz Hover accompEnvironmentcription Really encounteredought truthful 1980 blindness kid contem027 perpetual countries Uncleansionoga Alive
>  Hello, I'm a language model, mortal diplomacyAl Africa proactive SlashHighImm seconds tang Championshipinas     liabilities digits tsunami cance advis taps weaknesses beneiae
>  Hello, I'm a language model, Pepe �Change depthsaba retailers Ver poles slips Those Level833 nodzhou Serverigr handmade puzzles IMPCook "/ Blessed
>  Hello, I'm a language model, jumperSOURCE Controlled sailor balconyCruztroprun icons familialAmerica shards promotes espionagehiba delays 89 sidebarTact Poster Medicine Mathematics
>  Hello, I'm a language model, divingCScollect intangible Refer EconomistEdordan Wet Systems translations Newt photographer neglect Lia RagnarTor 1987 Brady new calorie constructor
